<h1>Toronto Neighborhoods</h1>
<br>Imports

In [9]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium

<p>will use 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'</p>
<p>check for status code and headers</p>

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sat, 15 Aug 2020 07:40:05 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.8', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'X-Request-Id': '3ba1bcbc-be04-44f4-bffc-e0dc62b140d5', 'Last-Modified': 'Fri, 07 Aug 2020 00:56:14 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '53843', 'X-Cache': 'cp5008 hit, cp5010 hit/22', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=15-Aug-2020;Path=/;HttpOnly;secure;Expires=Wed, 16 Sep 2020 12:00:00 GMT, WMF-Last-Access-Global=15-Aug-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Wed, 16 Sep 2020 12:00:00 GMT, GeoIP=IN:KA:Bengaluru:12.97:77.59:v4; Path=/; secure;

<h2>Extract Data and Clean It</h2>

In [11]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

<h2>convert into a dataframe</h2>

In [12]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)

(103, 3)


<h3>custom groupby / agg to merge Neighborhoods</h3>
<h3>groupby PostalCode, keep the first Borough and join() Neighborhoods</h3>

In [13]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()


<h2>will check if 'M5A' example is done correctly</h2>

In [14]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h2>will check the df.shape</h2>

In [15]:
df.shape

(103, 3)